In [1]:
!pip install Rouge

In [2]:
from google.colab import drive
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from rouge import Rouge

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pretrained_model_path = '/content/drive/My Drive/BioWordVec/bio_embedding_intrinsic'
pretrained_model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)

In [5]:
def preprocess_text(text):
    tokens = text.split()
    tokens = [word for word in tokens if word in pretrained_model.key_to_index]
    return tokens

def text_to_sequence(text, tokenizer, max_len):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=max_len, padding='post')[0]

In [6]:
df = pd.read_csv('merged_dataset_new.csv')

In [7]:
df['Output'] = "Primary Evidence: " +df['Primary_evidence_index'].fillna('') + ' ' " Secondary Evidences: "+ df['Secondary_evidence_index'].fillna('')


In [8]:
all_texts = df['Statement'].tolist() + df['Reference_data'].tolist() + df['Output'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

In [9]:
max_len = max([len(text.split()) for text in all_texts])
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = pretrained_model.vector_size

In [10]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in pretrained_model.key_to_index:
        embedding_matrix[i] = pretrained_model[word]

In [11]:
def text_to_sequence(text, tokenizer, max_len):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=max_len, padding='post')[0]

In [12]:
df['Statement_seq'] = df['Statement'].apply(lambda x: text_to_sequence(x, tokenizer, max_len))
df['Reference_data_seq'] = df['Reference_data'].apply(lambda x: text_to_sequence(x, tokenizer, max_len))
df['Output_seq'] = df['Output'].apply(lambda x: text_to_sequence(x, tokenizer, max_len))

In [13]:
X = np.array([np.hstack([s, r]) for s, r in zip(df['Statement_seq'].values.tolist(), df['Reference_data_seq'].values.tolist())])
y = df['Output'].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
y_train_seq = np.array([text_to_sequence(text, tokenizer, max_len) for text in y_train])
y_test_seq = np.array([text_to_sequence(text, tokenizer, max_len) for text in y_test])

In [ ]:
y_train_seq_cat = np.array([to_categorical(seq, num_classes=vocab_size) for seq in y_train_seq])
y_test_seq_cat = np.array([to_categorical(seq, num_classes=vocab_size) for seq in y_test_seq])

In [ ]:
### System Crashed After using all available RAM